# Stochastic Block Model experiments
As we have seen in previous experiments, Tangles do not outperform a simple SOE-kMeans baseline, when applied to euclidean data. This can be expected, as kMeans has a euclidean bias, which could help recover the original data. To see if tangles perform better on non-euclidean data, we will use a stochastic block model (SMB).

## Stochastic Block Model
An SMB is a graph model. For the simple models we use here, we have that each SMB consists of $k$ clusters with $n$ points each. There exists an edge between two points $v, w$ with probability $p$ if the points are in the same cluster, and an edge with probability $q$ if they are in different clusters.

In [1]:
import sys
sys.path.append("..")
from sklearn.metrics import normalized_mutual_info_score
from sklearn.cluster import SpectralClustering
from data_generation import generate_smb_data
from estimators import LandmarkTangles, SoeKmeans, OrdinalTangles
from questionnaire import Questionnaire
from tangles.loading import load_SBM
from tangles.cut_finding import kernighan_lin
from tangles.cost_functions import mean_edges_cut_cost
from functools import partial
import numpy as np
import random
import pandas as pd
import altair as alt
import networkx as nx
import math

np.random.seed(0)
random.seed(120384)

First, we are providing a function to easily evaluate tangles performance on SMB. We 
build an SMB, generate triplets from it via shortest path, run the baseline as well as tangles on it, and give back the results averaged over a number of runs.

In [2]:
def smb_performance(p, q, n=20, k=5, n_runs=1):
    """
    Tests the performance of tangles on an SMB. 
    Returns the score of the the tangles algorithm and of the baseline (SOE-kMeans) 
    average over n_runs.
    """
    tangles_scores = []
    baseline_scores = []
    for _ in range(n_runs):
        graph, ys = generate_smb_data(n=n, k=k, p=p, q=q)
        questionnaire = Questionnaire.from_graph(graph, density=0.1, verbose=False)

        t,r = questionnaire.to_bool_array()

        tangles = LandmarkTangles(8, verbose=0)
        tangles_labels = tangles.fit_predict(t, r)
        tangles_score = normalized_mutual_info_score(ys, tangles_labels)

        #
        soe_knn = SoeKmeans(2, 5)
        baseline_labels = soe_knn.fit_predict(t, r)
        baseline_score = normalized_mutual_info_score(ys, baseline_labels)

        baseline_scores.append(baseline_score)
        tangles_scores.append(tangles_score)

    return np.mean(tangles_scores), np.mean(baseline_scores)

A smoke test to see if everything runs and has expected results:

In [3]:
tangles_nmi, baseline_nmi = smb_performance(0.9, 0.1)
print(f"Tangles: {tangles_nmi}\nBaseline: {baseline_nmi}")

Tangles: 0.9347684918528811
Baseline: 0.8675564559194765


In the next step, we build a meshgrid over different parameter values and plot it in a heat map. This is similar to what [Solveig did in her SMB experiments](https://arxiv.org/abs/2006.14444) (but there she used [Kernighan-Lin](https://en.wikipedia.org/wiki/Kernighan–Lin_algorithm) to find cuts).

In [4]:
# meshgrid
p, q = np.meshgrid(np.arange(0.1, 1.0, 0.1), np.concatenate(
    ([0.01], np.arange(0.05, 0.5, 0.05))))

tangles_nmi_grid = np.zeros_like(p)
baseline_nmi_grid = np.zeros_like(q)
for i in range(p.shape[0]):
    for j in range(p.shape[1]):
        nmi_score, baseline_score = smb_performance(p[i, j], q[i, j])
        tangles_nmi_grid[i, j] = nmi_score
        baseline_nmi_grid[i, j] = baseline_score

tangles_nmi_df = pd.DataFrame(
    {'p': p.ravel(), 'q': q.ravel(), 'nmi': tangles_nmi_grid.ravel()})
baseline_nmi_df = pd.DataFrame(
    {'p': p.ravel(), 'q': q.ravel(), 'nmi': baseline_nmi_grid.ravel()})

tangles_chart = alt.Chart(tangles_nmi_df).mark_rect().encode(
    x=alt.X('p:O', axis=alt.Axis(title='p', format=".2")),
    y=alt.Y('q:O', axis=alt.Axis(title='q', format=".2"),
            sort=alt.EncodingSortField('q', order='descending'),
            ),
    color='nmi:Q'
).properties(title="Tangles").interactive()

baseline_chart = alt.Chart(baseline_nmi_df).mark_rect().encode(
    x=alt.X('p:O', axis=alt.Axis(title='p', format=".2")),
    y=alt.Y('q:O', axis=alt.Axis(title='q', format=".2"),
            sort=alt.EncodingSortField('q', order='descending'),
            ),
    color='nmi:Q'
).properties(title="Baseline").interactive()

KeyboardInterrupt: 

In [5]:
(baseline_chart | tangles_chart).show()

NameError: name 'baseline_chart' is not defined

The results we observe are a bit weird, we would expect Tangles to outperform the baseline. This seems to require further inquiries. We suspect that shortest path length might not be the best candidate, as out-of-cluster connections are quite prevalent, and due to there being just a lot more out-of-cluster nodes than in-cluster nodes, the possibility that a direct connection to an out-of-cluster node is existing is very high.

We use a simple MC-estimate to determine inner-path expected length, outer-path expected length and based on that, number of nodes that have an outer-cluster-node as nearest neighbour.

In [6]:
n = 20
k = 5
graph, ys = generate_smb_data(n=n, k=k, p=0.7, q=0.2)
paths = nx.floyd_warshall_numpy(graph)

inner_dists_list = []
outer_dists_list = []
num_wrong_neighbour_list = []

# average path length
for i in range(k):
    block_lower, block_upper = (i * n, (i + 1) * n)
    current_block_mask = np.ones_like(paths) == 0
    current_block_mask[block_lower:block_upper, block_lower:block_upper] = True
    inner_dists = paths[current_block_mask]
    outer_dists = paths[~current_block_mask]

    inner_dists_list.append(inner_dists.mean())
    outer_dists_list.append(outer_dists.mean())

    current_points = np.copy(paths[block_lower:block_upper, :])
    current_points[current_points == 0] = np.inf
    nearest_neighbours = np.argmin(current_points, axis=1)
    num_wrong_neighbour_list.append(np.sum(np.logical_or(nearest_neighbours <=
                                                         block_lower, nearest_neighbours >= block_upper))/n)

print(f"Average inner distance: {np.mean(inner_dists_list)}")
print(f"Average outer distance: {np.mean(outer_dists_list)}")
print(
    f"Average percentage of closest neighbour being outer cluster: {np.mean(num_wrong_neighbour_list)}")

Average inner distance: 1.2550000000000001
Average outer distance: 1.705833333333333
Average percentage of closest neighbour being outer cluster: 0.93


As we can see, the average inner vs outer distance is different, but not hugely. More concerning however, is the number of nodes with wrong cluster. We cannot expect to achieve any reasonable clustering with that (and it's suspicious that SOE-kMeans performs that well still, this might be due to induced bias, since we tell the algorithm the number of clusters).

We also find another peculiarity:

In [7]:
print(num_wrong_neighbour_list)

[0.65, 1.0, 1.0, 1.0, 1.0]


We see that all but the first cluster have their closest neighbour in another cluster. This is an artifact of the argmax. When multiple values in an array are the same, argmax always picks the first occurence. Additionally, for all clusters besides the first, there is a very high probability, that there is already a direct neighbour in the first cluster, for example with $q = 0.2$ and $n = 20$:

$$1 - (1-q)^n = 1 - 0.8^{20} = 0.0115$$ 

This causes argmax to almost always pick a random node from the first cluster.
We can fix this by providing a different argmin implementation.
Additionally, to really get a cluster structure, we need p and q a lot lower

In [8]:
def rand_argmin(x): 
    """Returns random indices of all elements of x with the smallest value
    over axis 1."""
    out = []
    for row in x:
        out.append(np.random.choice(np.flatnonzero(row == row.min())))
    return np.array(out)

n = 20
k = 5
graph, ys = generate_smb_data(n=n, k=k, p=0.7, q=0.2)
paths = nx.floyd_warshall_numpy(graph)

inner_dists_list = []
outer_dists_list = []
num_wrong_neighbour_list = []

# average path length
for i in range(k):
    block_lower, block_upper = (i * n, (i + 1) * n)
    current_block_mask = np.ones_like(paths) == 0
    current_block_mask[block_lower:block_upper, block_lower:block_upper] = True
    inner_dists = paths[current_block_mask]
    outer_dists = paths[~current_block_mask]

    inner_dists_list.append(inner_dists.mean())
    outer_dists_list.append(outer_dists.mean())

    current_points = np.copy(paths[block_lower:block_upper, :])
    current_points[current_points == 0] = np.inf
    nearest_neighbours = rand_argmin(current_points)
    num_wrong_neighbour_list.append(np.sum(np.logical_or(nearest_neighbours <=
                                                         block_lower, nearest_neighbours >= block_upper))/n)

print(f"Average inner distance: {np.mean(inner_dists_list)}")
print(f"Average outer distance: {np.mean(outer_dists_list)}")
print(
    f"Average percentage of closest neighbour being outer cluster: {np.mean(num_wrong_neighbour_list)}")

Average inner distance: 1.233
Average outer distance: 1.6954999999999998
Average percentage of closest neighbour being outer cluster: 0.63


Looks better already.

## Tangles as graph preprocessing
With that knowledge, we will take a look at using Tangles as preprocessing steps on graphs.

In [9]:
def tangles_smb_spectral_performance(p, q, n, k):
    graph, ys = generate_smb_data(n=n, k=k, p=p, q=q)
    questionnaire = Questionnaire.from_graph(graph, density=0.001, verbose=1)

    t,r = questionnaire.to_bool_array()

    # tangles
    tangles = LandmarkTangles(math.ceil(n/3), verbose=0)
    nmi = tangles.score(t, r, ys)

    # spectral
    sc = SpectralClustering(n_clusters=k, affinity='precomputed')
    baseline_labels = sc.fit_predict(X = nx.to_numpy_matrix(graph))
    baseline_score = normalized_mutual_info_score(ys, baseline_labels)
    return nmi, baseline_score

In [10]:
tangles_smb_spectral_performance(0.9, 0.1, 500, 2)

Generating questionnaire...
Generating question set...
Filling out questionnaire...


100%|██████████| 1000/1000 [00:00<00:00, 2507.15it/s]
/opt/homebrew/Caskroom/mambaforge/base/envs/tangles-thesis/lib/python3.9/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


(1.0, 1.0)

### Triplets vs KL

In [11]:
A, ys, G = load_SBM(block_sizes=[200, 200, 200, 200], p_in=0.7, p_out=0.1, seed=2)
cuts = kernighan_lin(A, nb_cuts=20, lb_f=0.1, seed=2, verbose=0, early_stopping=5)

In [12]:
questionnaire = Questionnaire.from_graph(G, density=0.001, verbose=1)

t,r = questionnaire.to_bool_array()
mean_edges_cost = partial(mean_edges_cut_cost, A, None)

# tangles
tangles = LandmarkTangles(40, verbose=0, cost_function=mean_edges_cost)
nmi = tangles.score(t, r, ys)

# with cuts
tangles_cuts = OrdinalTangles(40, verbose=0, cost_function=mean_edges_cost)
tangles_cuts.fit(cuts.T)
nmi_cuts = normalized_mutual_info_score(ys, tangles_cuts.labels_)
nmi, nmi_cuts

Generating questionnaire...
Generating question set...
Filling out questionnaire...


100%|██████████| 800/800 [00:00<00:00, 3956.20it/s]


(0.3744572399462906, 0.7745041314990786)

When one runs the above code, one can see that Tangles do not outperform simple cuts for 
preprocessing. 

The cut quality suffers in the triplet approach (which makes the Tangles clustering run slower). 
For the same run-time, this requires us to use less triplet cuts (making the clustering less accurate).